# Design of optimal toffoli gate in the GdW$_{30}$ system, via quantum optimal control.

This notebook uses both optimization on top of the Schrödinger equation (for the evolution operator), and on top of Lindblad's equation, in the presence of dissipation terms.

In [ ]:
try:
    get_ipython
    isnotebook = True
except:
    isnotebook = False

import os
import sys
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from qutip import *
from time import time as clocktime
from scipy.optimize import curve_fit
from qutip.qip.operations import toffoli

import nlopt

In [ ]:
import qocttools
import qocttools.models.GdW30 as GdW30
import qocttools.hamiltonians as hamiltonians
import qocttools.math_extra as math_extra
import qocttools.pulses as pulses
import qocttools.qoct as qoct
import qocttools.solvers as solvers
import qocttools.target as target

In [ ]:
qocttools.about()

In [ ]:
data = []

# Model

The model is defined by the Hamiltonian:

\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
where the time-independent part is given by:
\begin{equation}
        \hat{H}_0 = D\bigg[\hat{S}_z^2 - \frac{1}{3}S(S + 1)\bigg] + E[\hat{S}_x^2 - \hat{S}_y^2] - g\mu_B\hat{\vec{S}}\cdot\vec{H}
\end{equation}
and the time-dependent part is:
\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
The perturbation is a magnetic field:
\begin{equation}
        \hat{V} = -g\mu_B\hat{\vec{S}}\cdot\vec{H}_m 
\end{equation}

In this case:

* $S = 7/2$

* $D$ = 1281 MHz

* $E$ = 294 MHz

* $\vec{H} = (0.15, 0.0, 0.0)$ T

* $\vec{H}_m = (0, 0.001, 0.0)$ T

In [ ]:
S = 7/2 # spin
E = 294 # value in MHz
D = 1281 # value in MHz
dim = int(2*S + 1) #matrix dim

In [ ]:
H = np.array([0.15, 0, 0.0], dtype = float) #magnetic field in T
H_m = np.array([0, 0.001, 0], dtype = float) #only in presence of perturbation (T)

H0 = GdW30.hGdW30(D, E, H)
V = GdW30.vGdW30(H_m)

The code will work in the basis of eigenstates of the $\hat{H}_0$:
\begin{equation}
\hat{H}_0\vert\psi_i\rangle = \varepsilon_i \vert\psi_i\rangle
\end{equation}
The characteristic frequencies are the energy differences:
\begin{equation}
\omega_i = \varepsilon_{i+1}-\varepsilon_{i}.
\end{equation}
(we only consider the frequencies between neighbour energies).

In [ ]:
eigenvalues, eigenstates = H0.eigenstates()
H0 = H0.transform(eigenstates) - eigenvalues[0]
V = V.transform(eigenstates)
H0 = 0.5 * (H0 + H0.dag())
V = 0.5 *(V + V.dag())

In [ ]:
# In principle, we could just use V with the recently obtained eigenstates. Unfortunately, that
# would make the test results different in different computers, as the eigenstates can have different phases.
# Therefore, we use the V that was computed once, and stored in file "V". The commented code was used
# to generate and store V.
#V = V.transform(eigenstates)
#fileio.file_data_store("V", V, numformat = 'exp')
V = Qobj(fileio.file_data_read("V"))

In [ ]:
H = hamiltonians.hamiltonian(H0, [V])

In [ ]:
Sx = jmat(S, "x")
Sy = jmat(S, "y")
Sz = jmat(S, "z")
Sp = (Sx + (1j*Sy))
Sm = (Sx - (1j*Sy))

In [ ]:
# w holds the energy differences, i.e. the natural frequencies of the system.
w = np.zeros(dim-1)
tau = np.zeros(dim-1)
for i in range(dim - 1):
    w[i] = eigenvalues[i+1] - eigenvalues[i] #in MHz
    tau[i] = 2.0*np.pi/w[i]
    print("Period[{:d}] = {:f}".format(i, tau[i]))

We will set the maximum time to a number of times (20, in this case) the maximum characteristic period.

In [ ]:
T = tau.max()*20
print('T = {0:f} ns ({1:f} in code time)'.format(1000.0*T/(2.0*np.pi), T))

# Control function

The control function is parametrized with the Fourier expansion as follow:
\begin{equation}
    f(u, t) = \frac{1}{\sqrt{T}}u_0 + \frac{2}{\sqrt{T}}\sum_{k = 1}^{M}u_{2k}\cos(\omega_kt) + \frac{2}{\sqrt{T}}\sum_{k = 1}^{M}u_{2k + 1}\sin(\omega_kt),
\end{equation}
where $u_0\dots u_{2M + 1}$ are the control parameters. This way, we can compute the derivate respect any control parameter as
\begin{equation}
    \frac{\partial f}{\partial u_m}(u, t) = f(e_m, t),
\end{equation}
where $e_m$ is the set of parameters where all of them are zero except the m-th ane, that is equal to one.

This pulse parametrization is included in the typical_pulses.py file as pulse class.

We will define a time grid to do all the following calculations. We choose a small discretization to ensure that the gradient calculations are performed at good precision.

In [ ]:
time = math_extra.timegrid(H0, T, 5.0)
print("tau/dt = {:f}".format(tau.min()/(time[1]-time[0])))

Now, we define our parametrized pulse.

In [ ]:
maxhval = 20.0
kappa = maxhval*np.sqrt(T)/2.0

In [ ]:
max_frecuency = 8000 #in MHz
M = int(T*max_frecuency/(2*np.pi)) #fourier serie terms
print("Number of coefficients = {:d}".format(2*M+1))
#u = np.random.uniform(low = -1.0, high = 1.0, size = 2*M+1)
u = np.zeros(2*M+1)
u[1] = kappa/2.0
f = pulses.pulse("fourier", T, u = u)

In principle, we could start from the pulse that we have just defined, or set the parameters to a random value. We could also start from a pi-pulse tuned to the $\vert 6 \rangle \to \vert 7\rangle$ transition, which is the relevant transition in the Toffoli gate. In the following cell, if `pipulse = True`, that is exactly what is done.

However, what we will do in the testsuite is to read the parameters obtained in a previous run, that were already optimized. In this way the notebook runs faster.

In [ ]:
pipulse = False
if pipulse:
    reftransition = 6
    amp = np.pi/(T*abs(V.data[reftransition, reftransition+1]))
    frefvals = np.zeros_like(time)
    for i in range(time.size):
        frefvals[i] = pulses.pi_pulse(time[i], np.array([amp, w[reftransition], 0.0, T]))

    optu = f.fitparams(frefvals, time, u)
    u[:] = optu[:]
else:
    u = np.loadtxt('optimal-pulse.csv', delimiter = ' ')

# QOCT target function definition

Let us now attempt these transitions with QOCT pulses. Let us just try the Toffoli gate

In [ ]:
U_target = Qobj(toffoli().data)
U_0 = qeye(dim)

In [ ]:
def FU(U, u):
    return (1/dim**2) * (np.absolute(math_extra.frobenius_product(U_target, U)))**2

def dFdU(U, u):
    return (1/dim**2) * math_extra.frobenius_product(U_target,U)*U_target

# Nonlinear constraints

If needded, one can make the pulse to have a null value at initial time and T. In other words:
\begin{equation}
    f(u, 0) = f(u, T) = 0
\end{equation}
Knowing that $\sin(\omega_n 0) = \sin(\omega_n T) = 0$ and $\cos(\omega_n 0) = \cos(\omega_n T) = 1$ we need the control parameters to satisfy the following expresion:
\begin{equation}
    u_0 + 2\sum_{k = 1}^{M}u_{2k} = 0\ .
\end{equation}
We will also enforce that the average amplitude is zero,
\begin{equation}
\int_0^T\!{\rm d}t\;f(u, t) = 0
\end{equation}
which is enforced by:
\begin{equation}
u_0 = 0
\end{equation}

This can be done with the nonlinear equality constraints option in Nlopt package, having in mind that not all algorithms can use this function. In fact, of the algorithms that use gradient, only the LD_SLSQP seems to be able to handle generic constraints. The previous constraints are set by defining the functions:
\begin{equation}
g_1(u) = \sum_{k=1}^M u_{2k}
\end{equation}
and
\begin{equation}
g_2(u) = u_0
\end{equation}

In [ ]:
f.set_constraint('zero_boundaries')
f.set_constraint('zero_average')

Moreover, we will set inequality constraints: we want the coefficients of each individual term in the Fourier expansion to be not larger than some reasonable experimental value. For example, let us suppose that we do not want the amplitudes to be larger than 10 mT

# Gradient check

In [ ]:
tg = target.Target('evolutionoperator', Utarget = U_target)

In [ ]:
opt = qoct.Qoct(H, T, time.shape[0], tg, f, U_0,
                interaction_picture = True,
                solve_method= 'cfmagnus4')

In [ ]:
derqoct, dernum, error = opt.check_grad(u, 3)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))

# Optimization

In [ ]:
x, optval, res = opt.maximize(maxeval = 200, stopval = 0.99, verbose = True,
                              algorithm = nlopt.LD_SLSQP, 
                              upper_bounds = kappa * np.ones_like(u),
                              lower_bounds = -kappa * np.ones_like(u))

In [ ]:
f.set_parameters(x)
result = solvers.solve("cfmagnus4", H0, V, f, U_0, time, interaction_picture = True)
print("Final optimized fidelity = {:f}".format(FU(result.states[-1], x)))

In [ ]:
u = x.copy()

In [ ]:
if pipulse:
    np.savetxt('optimal-pulse.csv', x, delimiter = ' ')

# Fidelity check

In [ ]:
def transformedstates(rho0):
    rhoT = []
    for j in range(len(rho0)):
        rhoT.append(U_target * rho0[j] * U_target.dag())
    return rhoT

In [ ]:
rho_ini = math_extra.dmset(dim, dim+1)
rho_target = transformedstates(rho_ini)
D = len(rho_ini)
nrm = np.zeros(D)
for j in range(D):
    nrm[j] = (rho_ini[j].dag() * rho_ini[j]).tr()

In [ ]:
A = 0.10 * Sp

In [ ]:
rhoT=[]
for i in range (D):
    rhoT.append(solvers.solve('cfmagnus4', H0, V, f, rho_ini[i], time,
                returnQoutput = True, interaction_picture = True, cops=[A]).states[-1])

In [ ]:
print(math_extra.infidelity(U_target, rho_ini, rhoT))

# Optimization with Lindblad equation

In [ ]:
def Tfunction(rho):
    sum = 0
    for i in range(D):
        sum += (rho[i] * rho_target[i]).tr() / (D * nrm[i])
    return np.real(sum)

def Frhou(rho, u): # Total function to maximize
    # Input: Array de rho's y array de parametros
    # Output: F(rho,u)
    return Tfunction(rho) #+ Pfunction(u))

def dFdrho(rho, u):
    # Input: Array de rho's
    # Output: Array de dF/drho_i
    aux=[]
    for i in range (D):
        aux.append(0.5 * rho_target[i] / (D*nrm[i]))
    return aux

def dFdu(u, m):
    return 0

In [ ]:
tg = target.Target('generic', Fyu = Frhou, dFdy = dFdrho, dFdu = dFdu)

In [ ]:
H = hamiltonians.hamiltonian(H0, [V], [A])

In [ ]:
opt = qoct.Qoct(H, T, time.shape[0], tg, f, rho_ini,
                interaction_picture = True,
                solve_method = 'cfmagnus4')

In [ ]:
derqoct, dernum, error = opt.check_grad(u, 3)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))

In [ ]:
x, optval, res = opt.maximize(maxeval = 100, stopval = 0.99, verbose = True,
                              algorithm = nlopt.LD_SLSQP, 
                              upper_bounds = kappa * np.ones_like(u),
                              lower_bounds = -kappa * np.ones_like(u))
data.append(optval)

# Datafile

In [ ]:
with open("data", "w") as f:
    for i in data:
        f.write("{:.14e}\n".format(i))